In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload


# TO DO
## Structure
* move classes out of main

## Logging
* change destination for log files - /var/log?

## Configuration
* method for installing user config?

## Testing
* command line switches,

## Daemon
* implement daemon start/stop/restart features
* restart on crash


In [48]:
import logging
import logging.config

# parse arguments
import sys

# handle importing libraries based on config file
import importlib

# loop delay - sleep
from time import sleep

# clock
from datetime import datetime

# handle kill/interrupt singals
import signal

# shell utilities for manpiulating files
from shutil import copyfile, copyfileobj

##### PyPi Modules #####
# handle http requests
import requests

# rate limit the queries on the LMS server
from ratelimiter import RateLimiter

# create cache Path objects for managing temporary paths
from cachepath import CachePath, TempPath, Path

# lmsquery-fork for managing comms with lms server
import lmsquery

In [49]:
import epdlib
import Configuration

In [50]:
def do_exit(status=0):
    if 'TESTING' in globals():
        if TESTING:
            logging.fatal(f'Exit called, but ignored due to global var `TESTING` = {TESTING}')
    else:
        sys.exit(status)

In [51]:
def time_now():
    '''return current time in HH:MM format
        e.g. 04:44, 23:16
    Returns:
        `str`'''
    return datetime.now().strftime("%H:%M")

In [52]:
class SignalHandler(object):
    '''handle specific signals and allow graceful exiting while loop
        https://stackoverflow.com/a/31464349/5530152
    
    Signals Handled Gracefully:
        SIGINT
        SIGTERM
    Atributes:
        kill_now (bool) default: False
    '''
    kill_now = False
    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)
    
    def exit_gracefully(self, signum, frame):
        self.kill_now = True

In [53]:
class cache_art():
    '''download and cache album artwork from lms server
    
    Args:
        app_name(`str`): name of application - used to create /tmp/`app_name` directory
        
    Properties:
        app_name(`str`): name of application
        cache_path(:obj:`CachePath`): path to store cached artwork
        '''
    def __init__(self, app_name):
        self.app_name = app_name
        
    @property
    def app_name(self):
        '''name of application
            this is used to set the path in /tmp/`app_name`
            
        Sets:
            app_name(`str`): name of application
            cache_path(obj:`CachePath`): path to store cached artwork'''
        return self._app_name
    
    @app_name.setter
    def app_name(self, app_name):
        self._app_name = app_name
        self.cache_path = CachePath(app_name, dir=True)
    
    def cache_artwork(self, artwork_url=None, album_id=None):
        '''download and cache artwork as needed from lms server
        
        Args:
            artwork_url(`str`): URL of artwork on LMS server
            album_id(`str`): unique string that identifies artwork on LMS server
            
        Returns:
            (obj:`pathlib.Path`): path to album artwork'''
        if not artwork_url or not album_id:
            raise TypeError(f'missing required value: artwork_url: {artwork_url}, album_id: {album_id}')
                
        album_id = str(album_id)
        
        artwork_path = self.cache_path/(album_id+'.jpg')
        
        if artwork_path.exists():
            logging.debug(f'artwork previously cached')
            return artwork_path
        
        r = False
        try:
            r = requests.get(artwork_url, stream=True)
        except requests.exceptions.RequestException as e:
            logging.error(f'failed to fetch artwork at: {artwork_url}: {e}')
        
        if r:
            try:
                with open(artwork_path, 'wb') as outFile:
                    copyfileobj(r.raw, outFile)
                    logging.debug(f'wrote ablum artwork to: {artwork_path}')
            except (OSError, FileExistsError, ValueError) as e:
                logging.error(f'failed to write {artwork_path}')
        else:
            logging.error('failed to download album art due to previous errors')
            return None
        
        return artwork_path

    def clear_cache(self, force=False):
        '''clear the contents of the cache folder or wipe entirely
        
        Args:
            force(`bool`): False (default) - delete the files, True - delete entire directory and contents
            '''
        logging.debug(f'clearing previously downloaded files in {self.cache_path}')
        if force:
            logging.info(f'removing cache directory: {self.cache_path}')
            try:
                self.cache_path.rm()
            except Exception as e:
                logging.error(f'Error removing cach directory: {e}')
                return False
        else:
            try:
                self.cache_path.clear()
            except Exception as e:
                logging.error(f'Error removing files in cache d directory: {e}')
                return False
            
        return True

In [54]:
def main():
    import constants
    
    ########## CONSTANTS #########
    version = constants.version
    app_name = constants.app_name
    app_long_name = constants.app_long_name
    url = constants.url
    logging_cfg = Configuration.fullPath(constants.logging_cfg)
    default_cfg = constants.default_cfg
    system_cfg = constants.system_cfg
    user_cfg = Configuration.fullPath(constants.user_cfg)
    noartwork = Configuration.fullPath(constants.noartwork)
    
    waveshare = constants.waveshare
    layouts_file = constants.layouts
    
    keyError_fmt = 'KeyError: section [{}] is missing value: {}'
    configError_fmt = 'see section [{}] in config file {}'
    valError_fmt = ''
    
    ######### CONFIGURATION #########
    
    ##### SETUP LOGGING #####
    logging.config.fileConfig(logging_cfg)
#     logger = logging.getLogger(__name__)
    
    ##### PARSE COMMAND LINE ARGUMENTS #####
    options = Configuration.Options(sys.argv)
    # add options to the configuration object
    # for options that override the configuration file options, add in the format: 
    # dest=[[ConfigFileSectionName]]__[[Option_Name]]
    # options that are not in the configuraton file can be added using the format:
    # dest=commandline_option_name
    
    # set logging level
    options.add_argument('-l', '--log-level', ignore_none=True, 
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'], 
                         dest='main__log_level', 
                         help='set logging level: DEBUG, INFO, WARNING, ERROR')

    # alternative user_cfg file
    options.add_argument('-c', '--config', type=str, required=False, 
                         dest='user_cfg', ignore_none=True, default=user_cfg,
                         help=f'use the specified configuration file; default user config: {user_cfg}')
    
    # list servers - 
    options.add_argument('-s', '--list-servers', action='store_true', 
                         dest='list_servers',
                         default=False,
                         help='list servers and players found on local network and exit')
    
    # parse the command line options
    options.parse_args()

    ##### ACTION COMMANDLINE ARGS #####
    if options.options.list_servers:
        logging.error('FIXME - need to addd routine to list server info')
        do_exit(0)
        pass    
    
    # use an alternative user configuration file
    if 'user_cfg' in options.opts_dict:
        user_cfg = options.opts_dict['user_cfg']
    
    # read the configuration right most values overwrite left values
    # system overwrites default; user overwrites system
    config_file = Configuration.ConfigFile(config_files=[default_cfg, system_cfg, user_cfg])    
    
    # merge the configuration files and the command line options
    config = Configuration.merge_dict(config_file.config_dict, options.nested_opts_dict)
    
    # set root log level now
    ll = config['main']['log_level']
    logging.warning(f'log level is {ll}')
    logging.root.setLevel(ll)

    
    ##### OBJECTS #####
    # Signal handler for gracefully handling HUP/KILL signals
    sigHandler = SignalHandler()
    
    # LMS Query rate limiter wrapper - allow max of `max_calls` per `period` (seconds)
    lmsQuery_ratelimit = RateLimiter(max_calls=1, period=3)
    # create lms query object
    try:
        lms = lmsquery.LMSQuery(**config['lms_server'])
        if not lms.player_id:
            raise ValueError(keyError_fmt.format('lms_server', 'player_id'))
    except TypeError as e:
        logging.fatal(configError_fmt.format('lms_server', user_cfg))
        logging.fatal(f'Error: {e}')
        do_exit(0)
        
    except ValueError as e:
        logging.fatal(e)
        logging.fatal(f'locate server and player information with:\n$ {app_name} --list-servers')
        do_exit(0)
    
    # setup EPD Display
    try:
        epd = importlib.import_module('.'.join([waveshare, config['layouts']['display']]))
    except (KeyError) as e:
        logging.fatal(keyError_fmt.format('layouts', 'display'))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    except (ModuleNotFoundError) as e:
        logging.fatal(keyError_fmt.format('layouts', 'display'))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    
    # setup layouts for displaying content
    try:
        layouts = importlib.import_module(layouts_file)
        playing_layout_format = getattr(layouts, config['layouts']['now_playing'])
        stopped_layout_format = getattr(layouts, config['layouts']['stopped'])
        splash_layout_format = getattr(layouts, config['layouts']['splash'])
    except (ModuleNotFoundError) as e:
        logging.fatal(f'could not import layouts file: {layouts_file}')
        logging.fatal(e)
        do_exit(0)
    except KeyError as e:
        logging.fatal(keyError_fmt.format('layouts', e.args[0]))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    
    # set resolution for screen
    resolution = [epd.EPD_HEIGHT, epd.EPD_WIDTH]
    # sort to put longest dimension first for landscape layout
    resolution.sort(reverse=True)
    
    playing_layout = epdlib.Layout(layout=playing_layout_format, resolution=resolution)
    
    
    splash_layout = epdlib.Layout(layout=splash_layout_format, resolution=resolution)
    splash_layout.update_contents({'app_name': app_name,
                                   'version': version,
                                   'url': url})
    
    playing_layout = epdlib.Layout(layout=playing_layout_format, resolution=resolution)
    stopped_layout = epdlib.Layout(layout=stopped_layout_format, resolution=resolution)
    
    # scren objects for managing writing to screen
    screen = epdlib.Screen()
    screen.epd = epd.EPD()
    screen.initEPD()
    
    
    ########## EXECUTION ##########
    # Show splash screen
    logging.debug(f'starting up with this configuration: {config}')
    if config['main']['splash_screen']:
        # push the images in the layout to the screen
        screen.elements = splash_layout.blocks.values()
        # concat all the individual images
        screen.concat()
        # write the image
        screen.writeEPD()
    
    # refresh the screen when true
    refresh = False
    
    # vars for managing track ID, mode, album art
    nowplaying_id = None
    nowplaying_mode = None
    artwork_cache = cache_art(app_long_name)
    
    # loop forever waiting for a kill/interrupt signal
    try:
        while not sigHandler.kill_now:
            # clear the lms server response 
            response = None
        
            # use the ratelimiter to throttle requests
            with lmsQuery_ratelimit:
                try:
                    logging.debug(f'querying lms server for status of player: {lms.player_id}')
                    response = lms.now_playing()
                except requests.exceptions.ConnectionError as e:
                    logging.warning(f'Server could not find active player_id: {lms.player_id}')
                    logging.warning('is the specified player active?')
                    response = {'title': f'Could not connect player: {lms.player_id}',
                                'album': 'is player_id valid?',
                                'artist': 'see logs for more info',
                                'id': 'NONE', 'mode': 'ERROR - SEE LOGS'}
                    nowplaying_mode = response['mode']

                except KeyError as e:
                    logging.info(f'No playlist is active on player_id: {lms.player_id}')
                    response = {'title': 'No music is queued', 'id': 'NONE', 'mode': 'No Playlist'}
                    nowplaying_mode = response['mode']
                    
            if response:
                resp_id = response['id']
                resp_mode = response['mode']
                if resp_id != nowplaying_id or resp_mode != nowplaying_mode:
                    logging.info(f'track/mode change {resp_mode}')
                    nowplaying_id = resp_id
                    nowplaying_mode = resp_mode
                    
                    # add the path to the album art into the response
                    try:
                        artwork = artwork_cache.cache_artwork(response['artwork_url'], response['album_id'])
                    except KeyError as e:
                        logging.warning('no artwork available')
                        artwork = None
                    if not artwork:
                        artwork = noartwork
                    response['coverart'] = str(artwork)
                             
                    playing_layout.update_contents(response)
                    refresh = playing_layout
                else:
                    refresh = False                

            if nowplaying_mode != 'play' and screen.update.last_updated > 60:
                logging.debug('music appears to be paused')
                stopped_layout.update_contents({'time': time_now(), 'mode': nowplaying_mode})
                refresh = stopped_layout
                
            # only refresh if needed
            if refresh:
                logging.debug('refreshing display')
                screen.initEPD()
                screen.elements=refresh.blocks.values()
                screen.concat()
                screen.writeEPD()   
                refresh = False                            
                    
    
            sleep(0.5)
    finally:
        logging.info('cleaning up and wiping screen')
        artwork_cache.clear_cache()
        
#         screen.initEPD()
#         screen.clearEPD()
        
    return config

In [ ]:
TESTING = True
c = main()

Configuration:parse_args:154:WARNING - ignoring unknown options: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-bcd34a93-049b-4b54-82fc-01dad72fccde.json']
Configuration:parse_config:389:WARNING - /etc/slimpi.cfg does not exist
<ipython-input-54-453a9ca00f88>:main:75:WARNING - log level is DEBUG
Layout:__init__:82:DEBUG - no font specified yet
Layout:layout:199:DEBUG - calculating values from layout for resolution [600, 448]
Layout:_calculate_layout:232:DEBUG - ***title***
Layout:_check_keys:98:DEBUG - checking key/values
Layout:_check_keys:103:DEBUG - missing key: rand; adding and setting to False
Layout:_check_keys:103:DEBUG - missing key: inverse; adding and setting to False
Layout:_check_keys:103:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_keys:103:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:239:DEBUG - dimensions: (600, 256)
Layout:_calculate_layout:268:DEBUG - has explict position
Layout:_calculate_layout:2

Block:maxchar:322:DEBUG - maximum characters per line: 42
Block:text_formatter:365:DEBUG - formatted list:
 ['.']
Block:_text2image:382:DEBUG - random is set: True
Block:_text2image:395:DEBUG - text image dimensions: (4, 22)
Block:_text2image:416:DEBUG - setting random position within area
Layout:__init__:82:DEBUG - no font specified yet
Layout:layout:199:DEBUG - calculating values from layout for resolution [600, 448]
Layout:_calculate_layout:232:DEBUG - ***app_name***
Layout:_check_keys:98:DEBUG - checking key/values
Layout:_check_keys:103:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_keys:103:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:239:DEBUG - dimensions: (600, 269)
Layout:_calculate_layout:268:DEBUG - has explict position
Layout:_calculate_layout:270:DEBUG - abs_coordinates: (0, 0)
Layout:_scalefont:125:DEBUG - no max char set; using: 6
Layout:_scalefont:140:DEBUG - calculating font size
Layout:_scalefont:141:DEB

Layout:_check_keys:103:DEBUG - missing key: inverse; adding and setting to False
Layout:_check_keys:103:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_keys:103:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:239:DEBUG - dimensions: (240, 192)
Layout:_calculate_layout:249:DEBUG - has calculated position
Layout:_calculate_layout:270:DEBUG - abs_coordinates: (0, 0)
Layout:_calculate_layout:232:DEBUG - ***artist***
Layout:_check_keys:98:DEBUG - checking key/values
Layout:_check_keys:103:DEBUG - missing key: rand; adding and setting to False
Layout:_check_keys:103:DEBUG - missing key: inverse; adding and setting to False
Layout:_check_keys:103:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_keys:103:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:239:DEBUG - dimensions: (360, 96)
Layout:_calculate_layout:249:DEBUG - has calculated position
Layout:_calculate_layout:270:DEB

Layout:_calculate_layout:270:DEBUG - abs_coordinates: (0, 0)
Layout:_scalefont:125:DEBUG - no max char set; using: 6
Layout:_scalefont:140:DEBUG - calculating font size
Layout:_scalefont:141:DEBUG - using text: W W W ; maxchar: 6
Layout:_scalefont:142:DEBUG - using font at path: /home/pi/src/slimpi_epd/fonts/Ubuntu/Ubuntu-Regular.ttf
Layout:_scalefont:154:DEBUG - target X font dimension 510.0
Layout:_scalefont:155:DEBUG - target Y dimension: 39.199999999999996
Layout:_scalefont:173:DEBUG - Y target exceeded
Layout:_scalefont:185:DEBUG - test string: W W W ; dimensions for fontsize 41: (147, 40)
Layout:_set_images:298:DEBUG - ***time***)
Layout:_set_images:302:DEBUG - set text block
Block:area:97:DEBUG - maximum area: (600, 392)
Block:inverse:76:DEBUG - set inverse: True
Block:abs_coordinates:114:DEBUG - absolute coordinates: (0, 0)
Block:__init__:245:DEBUG - create TextBlock
Block:maxchar:322:DEBUG - maximum characters per line: 7
Block:text_formatter:365:DEBUG - formatted list:
 ['.']

In [18]:
c

{'main': {'splash_screen': 'True', 'log_level': 'DEBUG'},
 'lms_server': {'host': '', 'port': '9000', 'player_id': 'dc:a6:32:29:99:ef'},
 'layouts': {'display': 'epd5in83',
  'now_playing': 'threeRow',
  'stopped': 'clock',
  'splash': 'splash'},
 '__cmd_line': {'list_servers': False}}